In [1]:
%pwd

'/content'

In [2]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
# !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu118.html

2.1.0+cu118
Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.8/887.8 kB 63.9 MB/s eta 0:00:00


In [3]:
# !pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.5 MB/s eta 0:00:00


In [4]:
import torch_geometric

In [5]:
class arguments:
  def __init__(self, seed = 10):
    self.seed = seed

In [6]:
args = arguments()

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
import torch_geometric
import os.path as osp

import torch
import torch.nn.functional as F

from torch_geometric.datasets import Planetoid
from torch_geometric.explain import Explainer, GNNExplainer
from torch_geometric.nn import GCNConv

import random

import torch
from torch_geometric.explain import Explainer, GNNExplainer
from torch_geometric.data import DataLoader
from torch_geometric.datasets import Planetoid
import networkx as nx
from sklearn.metrics import pairwise_distances

from sklearn.metrics import jaccard_score
from sklearn.metrics import pairwise_distances
from scipy.stats import wasserstein_distance, entropy
import numpy as np
import pandas as pd


from scipy.stats import wasserstein_distance, entropy
import numpy as np
import pandas as pd

from scipy.stats import wasserstein_distance, entropy
import numpy as np
import pandas as pd

from torch_geometric.utils import to_dense_adj
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool

In [9]:
# seeds
# random seed setting
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)

In [10]:
# model that we will use to load the weights
class GCNLayer(nn.Module):
    def __init__(self, in_channels, out_channels, with_bias=True):
        super(GCNLayer, self).__init__()
        self.conv = GCNConv(in_channels, out_channels, bias=with_bias)

    def forward(self, x, edge_index):
        x = self.conv(x, edge_index)
        return F.relu(x)

class GCNModel(nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes, num_layers=2,
                 dropout=0.5, with_bn=False):
        super(GCNModel, self).__init__()

        self.layers = nn.ModuleList([])
        self.num_layers = num_layers

        if with_bn:
            self.bns = nn.ModuleList([])

        # Input layer
        self.layers.append(GCNLayer(num_features, hidden_channels))
        if with_bn:
            self.bns.append(nn.BatchNorm1d(hidden_channels))

        # Hidden layers
        for _ in range(num_layers - 2):
            self.layers.append(GCNLayer(hidden_channels, hidden_channels))
            if with_bn:
                self.bns.append(nn.BatchNorm1d(hidden_channels))

        # Output layer
        self.layers.append(GCNLayer(hidden_channels, num_classes))

        self.dropout = dropout
        self.with_bn = with_bn

    def forward(self, x, edge_index):
        for i, layer in enumerate(self.layers):
            x = layer(x, edge_index)
            if i != self.num_layers - 1:
                x = self.bns[i](x) if self.with_bn else x
                x = F.relu(x)
                x = F.dropout(x, p=self.dropout, training=self.training)

        return F.log_softmax(x, dim=1)



In [11]:
# training a Full model for Cora and Citeseer - Storing the weights - Can be done in a different file too

In [99]:

class Explanations:
    def __init__(self, model1, model2, dataset, nodes_to_explain, threshold_val):
        self.model1 = model1.to(device)
        self.model2 = model2.to(device)
        self.dataset_name = dataset
        self.nodes_to_explain = nodes_to_explain
        self.data = self.load_data().to(device)
        self.explanations1 = {}
        self.explanations2 = {}
        self.threshold_val = threshold_val

    def load_data(self):
        if self.dataset_name == 'cora':
            dataset = Planetoid(root='data/Cora', name='Cora')
        elif self.dataset_name == 'citeseer':
            dataset = Planetoid(root='data/CiteSeer', name='CiteSeer')
        else:
            raise ValueError("Invalid dataset name. Supported datasets: 'cora', 'citeseer'")
        return dataset[0]

    def generate_explanation(self, node_index):
        explainer1 = Explainer(
            model=self.model1,
            algorithm=GNNExplainer(epochs=200),
            explanation_type='model',
            node_mask_type='object',
            edge_mask_type='object',
            model_config=dict(
                mode='multiclass_classification',
                task_level='node',
                return_type='log_probs',
            ),
            threshold_config = dict(threshold_type = 'topk', value = self.threshold_val)
        )

        explainer2 = Explainer(
            model=self.model2,
            algorithm=GNNExplainer(epochs=200),
            explanation_type='model',
            node_mask_type='object',
            edge_mask_type='object',
            model_config=dict(
                mode='multiclass_classification',
                task_level='node',
                return_type='log_probs',
            ),
            threshold_config = dict(threshold_type = 'topk', value = self.threshold_val)
        )

        explanation1 = explainer1(self.data.x, self.data.edge_index, index=node_index)
        explanation2 = explainer2(self.data.x, self.data.edge_index, index=node_index)

        # print(explanation1)
        # explanation1.visualize_graph()
        # explanation2.visualize_graph()
        # print("sizes of nodemask, edgemask",explanation1.node_mask.size(),explanation1.edge_mask.size() )

        self.explanations1[node_index] = explanation1
        self.explanations2[node_index] = explanation2

    def jaccard_similarity(self, explanation1, explanation2):
      # Step 1: Extract node and edge sets from the masks
      nodes_set1 = set(explanation1.node_mask.nonzero().flatten().tolist())
      edges_set1 = set(explanation1.edge_mask.nonzero().flatten().tolist())

      nodes_set2 = set(explanation2.node_mask.nonzero().flatten().tolist())
      edges_set2 = set(explanation2.edge_mask.nonzero().flatten().tolist())
      print("Node Sets of explanation", nodes_set1,nodes_set2)
      print("edge sets of explanation", edges_set1,edges_set2)

      # Step 2: Calculate Jaccard Similarity
      if len(nodes_set1.union(nodes_set2))==0:
        jaccard_similarity_nodes =0
      else:
        jaccard_similarity_nodes = len(nodes_set1.intersection(nodes_set2)) / len(nodes_set1.union(nodes_set2))

      if len(edges_set1.union(edges_set2))==0:
        jaccard_similarity_edges = 0
      else:
        jaccard_similarity_edges= len(edges_set1.intersection(edges_set2)) / len(edges_set1.union(edges_set2))

      return jaccard_similarity_nodes, jaccard_similarity_edges

    def cosine_similarity(self, explanation1, explanation2):
        # Step 1: Flatten the node and edge masks
        node_mask1 = explanation1.node_mask.view(-1)
        edge_mask1 = explanation1.edge_mask.view(-1)

        node_mask2 = explanation2.node_mask.view(-1)
        edge_mask2 = explanation2.edge_mask.view(-1)

        # Step 2: Concatenate the flattened masks into vectors
        vector1 = torch.cat((node_mask1, edge_mask1))
        vector2 = torch.cat((node_mask2, edge_mask2))

        # print("vector_size",vector1.size())

        # Step 3: Calculate cosine similarity
        similarity = F.cosine_similarity(vector1, vector2, dim=0)

        return similarity.item()

    def compare_explanations(self, node_index):
        # print("self.ex1", self.explanations1)
        # print("self.ex2", self.explanations2)
        print(f"============comparing explanatin for {node_index} ================")
        explanation1 = self.explanations1[node_index]
        explanation2 = self.explanations2[node_index]

        jaccard_similarity_nodes, jaccard_similarity_edges = self.jaccard_similarity(explanation1, explanation2)
        cosine_similarity = self.cosine_similarity(explanation1, explanation2)
        del self.explanations1[node_index]
        del self.explanations2[node_index]
        return jaccard_similarity_nodes, jaccard_similarity_edges, cosine_similarity

    def run_comparison(self):
        results = {'jaccard_similarity_nodes': [], 'jaccard_similarity_edges': [], 'cosine_similarity': []}

        for node_index in self.nodes_to_explain:
            self.generate_explanation(node_index)
            jaccard_similarity_nodes, jaccard_similarity_edges, cosine_similarity = self.compare_explanations(node_index)

            results['jaccard_similarity_nodes'].append(jaccard_similarity_nodes)
            results['jaccard_similarity_edges'].append(jaccard_similarity_edges)
            results['cosine_similarity'].append(cosine_similarity)


        #df = pd.DataFrame(results)
        return results

In [13]:

# # function defined to load model for different methods
# def load_model_method(path):
# '''creates a gcn_model and loads the weights and returns a loaded model'''
#   # Create an instance of the GCNModel
#   gcn_model = GCNModel(num_features = 1433, hidden_channels = 256, num_classes = 7, num_layers = 2, dropout=0.5, with_bn = False)
#   state_dict = torch.load(path)
#   for key in list(state_dict.keys()):
#       if key == 'layers.0.weight':
#         newkey = 'layers.0.conv.lin.weight'
#         state_dict[newkey] = state_dict.pop(key)
#         state_dict[newkey] = state_dict[newkey].T
#       elif key == 'layers.0.bias':
#         newkey = 'layers.0.conv.bias'
#         state_dict[newkey] = state_dict.pop(key)
#       elif key == 'layers.1.weight':
#         newkey = 'layers.1.conv.lin.weight'
#         state_dict[newkey] = state_dict.pop(key)
#         state_dict[newkey] = state_dict[newkey].T
#       elif key == 'layers.1.bias':
#         newkey = 'layers.1.conv.bias'
#         state_dict[newkey] = state_dict.pop(key)

#   gcn_model.load_state_dict(state_dict)
#   return gcn_model

In [14]:
# path = ("/content/sample_data/model_cora_0.5_1_trans.pt")
# gcn_model = load_model_method(path)

In [15]:
# bench_model = GCNModel(num_features = 1433, hidden_channels = 256, num_classes = 7, num_layers = 2, dropout=0.5, with_bn = False)
# path = ""
# state_dict = torch.load(path)
# model1.load_state_dict(state_dict)

In [16]:
# # Example usage:
# # Assuming you have two models (model1, model2) and a list of nodes to explain (nodes_to_explain)
# nodes_to_explain = [1708, 1709, 1710, 1711, 1712]
# explanations = Explanations(model1, gcn_model, 'cora', nodes_to_explain)
# results_df = explanations.run_comparison()
# print(results_df)

In [17]:
# ex1 = explanations.explanations1
# ex1

In [18]:
# ex2 = explanations.explanations2
# ex2

In [19]:
# ex1[1708].visualize_feature_importance(top_k =10)

In [20]:
# torch.sum(ex1[1708].get('node_mask') ==1)

In [21]:
# ex2[1709].visualize_graph('subgraph.png')

In [22]:
# ex2[1709]

In [23]:
# (ex2[1709].node_mask[ex2[1709].node_mask > 0])

In [24]:
# sub =ex1[1709].get_explanation_subgraph()

In [25]:
# sub

In [26]:
# sub.target

In [27]:
# def calculate_jaccard_similarity(explanation1, explanation2):
#     # Step 1: Extract node and edge sets from the masks
#     nodes_set1 = set(explanation1.node_mask.nonzero().flatten().tolist())
#     edges_set1 = set(explanation1.edge_mask.nonzero().flatten().tolist())
#     print(edges_set1)

#     nodes_set2 = set(explanation2.node_mask.nonzero().flatten().tolist())
#     edges_set2 = set(explanation2.edge_mask.nonzero().flatten().tolist())
#     print(edges_set2)

#     # Step 2: Calculate Jaccard Similarity
#     jaccard_similarity_nodes = len(nodes_set1.intersection(nodes_set2)) / len(nodes_set1.union(nodes_set2))
#     jaccard_similarity_edges = len(edges_set1.intersection(edges_set2)) / len(edges_set1.union(edges_set2))

#     return jaccard_similarity_nodes, jaccard_similarity_edges

In [28]:
import torch
import torch.nn.functional as F

# def calculate_cosine_similarity(explanation1, explanation2):
#           # Step 1: Flatten the node and edge masks
#         node_mask1 = explanation1.node_mask.view(-1)
#         edge_mask1 = explanation1.edge_mask.view(-1)

#         node_mask2 = explanation2.node_mask.view(-1)
#         edge_mask2 = explanation2.edge_mask.view(-1)

#         # Step 2: Concatenate the flattened masks into vectors
#         vector1 = torch.cat((node_mask1, edge_mask1))
#         vector2 = torch.cat((node_mask2, edge_mask2))

#         # Step 3: Calculate cosine similarity
#         similarity = F.cosine_similarity(vector1, vector2, dim=0)

#         return similarity.item()


In [29]:
# import networkx as nx
# def GED(explanation1, explanation2):

#   # Assuming you have graphs represented by edge indices
#   graph1 = nx.Graph(explanation1.edge_index.t().numpy())
#   graph2 = nx.Graph(explanation2.edge_index.t().numpy())

#   # Calculate Graph Edit Distance
#   ged = nx.graph_edit_distance(graph1, graph2)
#   print("Graph Edit Distance:", ged)

In [30]:
# calculate_jaccard_similarity(ex1[1709], ex2[1709])

In [31]:
# calculate_cosine_similarity(ex1[1709], ex2[1709])

In [32]:
# import torch
# from torch_geometric.utils import to_edge_index,to_torch_coo_tensor

# # Assuming adjacency_matrix is your adjacency matrix tensor
# adjacency_matrix = torch.tensor([[0, 1, 1], [1, 0, 0], [1, 0, 0]])

# adj = to_torch_coo_tensor(adjacency_matrix)


# # Convert adjacency matrix to edge index
# #edge_index = to_edge_index(adjacency_matrix)

# # Print the edge index
# print(edge_index)

In [33]:
# i = torch.tensor([[0, 1, 1],[2, 0, 2]])
# v = torch.tensor([3, 4, 5], dtype=torch.float32)
# s = torch.sparse_coo_tensor(i, v, [2, 4])

In [34]:
# s.to_dense()

In [35]:
# from torch_geometric.utils import to_dense_adj
# edge_index = torch.tensor([[0, 1, 1, 2, 2, 3],
#                            [1, 0, 2, 1, 3, 2]])
# adj = to_dense_adj(edge_index)
# adj

In [36]:
# dataset = Planetoid(root='data/Cora', name='Cora')
# dataset[0]

In [37]:
# def generate_random_testset(data, seedlist,samplesize):
#   indices = torch.nonzero(data.test_mask).squeeze().tolist()
#   out = {}
#   for aseed in seedlist:
#     np.random.seed(aseed)
#     alist = np.random.choice(indices, size=samplesize)
#     out[aseed] = alist

#   return out

In [38]:
# generate_random_testset(dataset[0], [15,1,25], 5)

In [100]:
class experiment2():
  def __init__(self, cora_data, citeseer_data, args, seedlist, sample_per_seed, explanation_threshold_list, model_cora_bench, model_citeseer_bench):
    self.args = args
    self.seeds = {}
    self.sample_per_seed = sample_per_seed
    self.seedlist = seedlist
    self.cora = cora_data
    self.citeseer = citeseer_data
    self.threshold_list = explanation_threshold_list
    self.seeds["cora"] = self.generate_random_testset(self.cora,seedlist, sample_per_seed)
    self.seeds["citeseer"] = self.generate_random_testset(self.citeseer,seedlist, sample_per_seed)
    self.model_cora_bechmark = model_cora_bench
    self.model_citeseer_bechmark = model_citeseer_bench
    self.out = {} # method : {threshold: {seeds: meanvalue}}

    self.results_store = datastorage(self.threshold_list)


  def generate_random_testset(self, data, seedlist,samplesize):
    indices = torch.nonzero(data.test_mask).squeeze().tolist()
    out = {}
    for aseed in seedlist:
      np.random.seed(aseed)
      alist = np.random.choice(indices, size=samplesize)
      out[aseed] = alist

    return out

  def load_model_method(self,path, dataname = "cora"):
    '''creates a gcn_model and loads the weights and returns a loaded model'''
      # Create an instance of the GCNModel
    if dataname == 'cora':
      gcn_model = GCNModel(num_features = 1433, hidden_channels = 256, num_classes = 7, num_layers = 2, dropout=0.5, with_bn = False)
    elif dataname == "citeseer":
      gcn_model = GCNModel(num_features = 3703, hidden_channels = 256, num_classes = 6, num_layers = 2, dropout=0.5, with_bn = False)
    state_dict = torch.load(path)
    for key in list(state_dict.keys()):
        if key == 'layers.0.weight':
          newkey = 'layers.0.conv.lin.weight'
          state_dict[newkey] = state_dict.pop(key)
          state_dict[newkey] = state_dict[newkey].T
        elif key == 'layers.0.bias':
          newkey = 'layers.0.conv.bias'
          state_dict[newkey] = state_dict.pop(key)
        elif key == 'layers.1.weight':
          newkey = 'layers.1.conv.lin.weight'
          state_dict[newkey] = state_dict.pop(key)
          state_dict[newkey] = state_dict[newkey].T
        elif key == 'layers.1.bias':
          newkey = 'layers.1.conv.bias'
          state_dict[newkey] = state_dict.pop(key)

    gcn_model.load_state_dict(state_dict)
    return gcn_model

  def run_experiment(self, pathlist):

    # load the model
    for key, value in pathlist.items():
      dataname,method,compression,model_seed = key
      paths = value
      # collect all nodes for different seeds
      nodes_to_explain = []
      for aseed in self.seedlist:
        nodes_to_explain.extend(self.seeds[dataname][aseed].tolist())

      for path in paths:
        #path = ("/content/sample_data/model_cora_0.5_1_trans.pt")
        gcn_model = self.load_model_method(path, dataname)

        for thresh in self.threshold_list:
          if dataname =="cora":
            print("Cora nodes: ", nodes_to_explain, "Threshold", thresh )
            #nodes_to_explain = self.seeds[dataname][aseed]
            explanations = Explanations(self.model_cora_bechmark, gcn_model, 'cora', nodes_to_explain,thresh)
            results = explanations.run_comparison()
            print("Cora ==> ", results)
            self.store(results,[dataname, method, compression, model_seed, thresh])
          elif dataname == "citeseer":
            print("Citeseer nodes: ", nodes_to_explain, "Threshold", thresh )
            # nodes_to_explain = self.seeds[dataname][aseed]
            explanations = Explanations(self.model_citeseer_bechmark, gcn_model, 'citeseer', nodes_to_explain,thresh)
            results = explanations.run_comparison()
            print("Citeseer ==> ", results)
            self.store(results,[dataname, method, compression, model_seed, thresh])

  def store(self,results,details):
    # details data structure : (dataset/method/compression/seed)
    self.results_store.update(results,details)

In [101]:
import pandas as pd
class datastorage():
  def __init__(self, theshold_list):
    self.methods = ["random","herding","kcenter","kmeans", "gcond","sfgc", "one-step"]
    self.threshold = theshold_list
    self.data = ["cora", "citeseer"]
    self.compression =  [0.125,0.25,0.5]

    # create tables 3seeds - 1,15,85 x 3metrics - Jac_node, Jac_edge, cosine
    self.jac_node_1 = self.create_table()
    self.jac_node_15 = self.create_table()
    self.jac_node_85 = self.create_table()

    self.jac_edge_1 = self.create_table()
    self.jac_edge_15 = self.create_table()
    self.jac_edge_85 = self.create_table()

    self.cosine_1 = self.create_table()
    self.cosine_15 = self.create_table()
    self.cosine_85 = self.create_table()


  def create_table(self):
    col_idx = pd.MultiIndex.from_product([self.methods,self.threshold],names = ["methods","threshold"])
    row_idx = pd.MultiIndex.from_product([self.data,self.compression],names = ["Dataset","Compression ratio"])
    df = pd.DataFrame(columns=col_idx, index=row_idx)
    df = df.fillna(0)
    return df

  def update(self, results, details):
    # details datastructure : (dataset/method/compression/seed)
    dataset, method, compression, seed, threshold = details
    print("Keys", (dataset, compression), (method, threshold))
    print("Shape of mean",np.mean(results["jaccard_similarity_nodes"]).shape)
    #print("Shape of jacard",self.jac_node_1.loc[(dataset, compression), (method, threshold)])

    if seed==1:
      self.jac_node_1.loc[(dataset,compression),(method,threshold)] = np.mean(results["jaccard_similarity_nodes"])
      self.jac_edge_1.loc[(dataset,compression),(method,threshold)] = np.mean(results["jaccard_similarity_edges"])
      self.cosine_1.loc[(dataset,compression),(method,threshold)] = np.mean(results["cosine_similarity"])

    if seed==15:
      self.jac_node_15.loc[(dataset,compression),(method,threshold)] = np.mean(results["jaccard_similarity_nodes"])
      self.jac_edge_15.loc[(dataset,compression),(method,threshold)] = np.mean(results["jaccard_similarity_edges"])
      self.cosine_15.loc[(dataset,compression),(method,threshold)] = np.mean(results["cosine_similarity"])

    if seed == 85:
      self.jac_node_85.loc[(dataset,compression),(method,threshold)] = np.mean(results["jaccard_similarity_nodes"])
      self.jac_edge_85.loc[(dataset,compression),(method,threshold)] = np.mean(results["jaccard_similarity_edges"])
      self.cosine_85.loc[(dataset,compression),(method,threshold)] = np.mean(results["cosine_similarity"])

  def display_save(self):
    # Create a new dataframe 'average' for jacard nodes
    average_jac_nodes = self.create_table()
    for col in average_jac_nodes.columns:
      average_jac_nodes[col] = (self.jac_node_1[col] + self.jac_node_15[col] + self.jac_node_85[col]) / 3

    # Create a new dataframe 'average' for jacard edges
    average_jac_edges = self.create_table()
    for col in average_jac_edges.columns:
      average_jac_edges[col] = (self.jac_edge_1[col] + self.jac_edge_15[col] + self.jac_edge_85[col]) / 3

    # Create a new dataframe 'average' for cosine
    average_cosine = self.create_table()
    for col in average_cosine.columns:
      average_cosine[col] = (self.cosine_1[col] + self.cosine_15[col] + self.cosine_85[col]) / 3

    print("Jacard Nodes Similarity")
    print(average_jac_nodes)
    average_jac_nodes.to_csv("Avg_Jacard_nodes_similarity")
    print("Jacard Edges Similarity")
    print(average_jac_edges)
    average_jac_edges.to_csv("Avg_Jacard_edges_similarity")
    print("Cosine Similarity - Overall")
    print(average_cosine)
    average_cosine.to_csv("Avg_Cosine_similarity")

    output_folder = 'rawFiles'
    os.makedirs(output_folder, exist_ok=True)
    self.jac_node_1.to_csv("rawFiles/jac_node_1")
    self.jac_node_15.to_csv("rawFiles/jac_node_15")
    self.jac_node_85.to_csv("rawFiles/jac_node_85")

    self.jac_edge_1.to_csv("rawFiles/jac_edge_1")
    self.jac_edge_15.to_csv("rawFiles/jac_edge_15")
    self.jac_edge_85.to_csv("rawFiles/jac_edge_85")

    self.cosine_1.to_csv("rawFiles/cosine_1")
    self.cosine_15.to_csv("rawFiles/cosine_15")
    self.cosine_85.to_csv("rawFiles/cosine_85")







In [103]:
# load data
dataset = Planetoid(root='data', name= "cora")
cora_data = dataset[0]
dataset = Planetoid(root='data', name= "citeseer")
citeseer_data = dataset[0]

# Experiment parameters
seedlist = [1,85,105]
sample_per_seed = 10
explanation_threshold_list = [5,10,15]

# load bench - cora
model_cora_bench = GCNModel(num_features = 1433, hidden_channels = 256, num_classes = 7, num_layers = 2, dropout=0.5, with_bn = False)
state_dict = torch.load("/content/sample_data/model_cora.pt")
model_cora_bench.load_state_dict(state_dict)

# load bench - citeseer
model_citeseer_bench = GCNModel(num_features = 3703, hidden_channels = 256, num_classes = 6, num_layers = 2, dropout=0.5, with_bn = False)
state_dict = torch.load("/content/sample_data/model_citeseer.pt")
model_citeseer_bench.load_state_dict(state_dict)

exp = experiment2(cora_data, citeseer_data, args, seedlist, sample_per_seed, explanation_threshold_list, model_cora_bench, model_citeseer_bench)

In [104]:
#dataname,method,compression,model_seed
pathlist = {("cora","gcond",0.5,1): ["/content/sample_data/model_cora_0.5_1_trans.pt"], ("citeseer","gcond",0.25,1):[ "/content/sample_data/model_citeseer_0.25_1_trans.pt" ]}
exp.run_experiment(pathlist)

Cora nodes:  [1745, 1943, 2616, 1780, 2475, 2613, 2423, 2353, 2555, 2668, 2070, 1868, 2617, 2416, 2271, 2617, 2272, 2303, 2257, 2683, 2412, 2337, 2498, 1781, 1936, 2508, 2412, 2256, 1884, 2640] Threshold 5
============comparing explanatin for 1745 ================
Node Sets of explanation {0, 449, 650, 1199, 1136, 2334} {0, 2596, 101, 1589, 1751, 1499}
edge sets of explanation {228, 4010, 7052, 5680, 3194} {7075, 5935, 10328, 10330, 445}
============comparing explanatin for 1943 ================
Node Sets of explanation {0, 1413, 1542, 872, 1939, 1943} {0, 1413, 1542, 1943, 1176, 1340}
edge sets of explanation {3466, 7916, 1038, 7926, 4607} {3489, 7924, 5046, 3127, 927}
============comparing explanatin for 2616 ================
Node Sets of explanation {64, 0, 1737, 1169, 2616, 1209} {64, 0, 1737, 1169, 2616, 1209}
edge sets of explanation {10370, 10371, 7660, 5395, 4575} {10371, 6988, 241, 242, 4575}
============comparing explanatin for 1780 ================
Node Sets of explanation {

In [105]:
exp.results_store.display_save()

Jacard Nodes Similarity
methods                    random           herding           kcenter       \
threshold                      5    10   15      5    10   15      5    10   
Dataset  Compression ratio                                                   
cora     0.125                0.0  0.0  0.0     0.0  0.0  0.0     0.0  0.0   
         0.250                0.0  0.0  0.0     0.0  0.0  0.0     0.0  0.0   
         0.500                0.0  0.0  0.0     0.0  0.0  0.0     0.0  0.0   
citeseer 0.125                0.0  0.0  0.0     0.0  0.0  0.0     0.0  0.0   
         0.250                0.0  0.0  0.0     0.0  0.0  0.0     0.0  0.0   
         0.500                0.0  0.0  0.0     0.0  0.0  0.0     0.0  0.0   

methods                         kmeans  ...          gcond            \
threshold                    15     5   ...   15        5         10   
Dataset  Compression ratio              ...                            
cora     0.125              0.0    0.0  ...  0.0  0.00000

In [106]:
exp.results_store.jac_node_1

methods                    random       herding       kcenter       kmeans  \
threshold                      5  10 15      5  10 15      5  10 15     5    
Dataset  Compression ratio                                                   
cora     0.125                  0  0  0       0  0  0       0  0  0      0   
         0.250                  0  0  0       0  0  0       0  0  0      0   
         0.500                  0  0  0       0  0  0       0  0  0      0   
citeseer 0.125                  0  0  0       0  0  0       0  0  0      0   
         0.250                  0  0  0       0  0  0       0  0  0      0   
         0.500                  0  0  0       0  0  0       0  0  0      0   

methods                     ...        gcond                     sfgc        \
threshold                   ... 15        5         10        15   5  10 15   
Dataset  Compression ratio  ...                                               
cora     0.125              ...  0  0.000000  0.000000  0.000000    0  0  0   
         0.250              ...  0  0.000000  0.000000  0.000000    0  0  0   
         0.500              ...  0  0.525758  0.700046  0.772700    0  0  0   
citeseer 0.125              ...  0  0.000000  0.000000  0.000000    0  0  0   
         0.250              ...  0  0.683968  0.805343  0.850326    0  0  0   
         0.500              ...  0  0.000000  0.000000  0.000000    0  0  0   

methods                    one-step        
threshold                        5  10 15  
Dataset  Compression ratio                 
cora     0.125                    0  0  0  
         0.250                    0  0  0  
         0.500                    0  0  0  
citeseer 0.125                    0  0  0  
         0.250                    0  0  0  
         0.500                    0  0  0  

[6 rows x 21 columns]

In [108]:
exp.results_store.jac_edge_1

methods                    random       herding       kcenter       kmeans  \
threshold                      5  10 15      5  10 15      5  10 15     5    
Dataset  Compression ratio                                                   
cora     0.125                  0  0  0       0  0  0       0  0  0      0   
         0.250                  0  0  0       0  0  0       0  0  0      0   
         0.500                  0  0  0       0  0  0       0  0  0      0   
citeseer 0.125                  0  0  0       0  0  0       0  0  0      0   
         0.250                  0  0  0       0  0  0       0  0  0      0   
         0.500                  0  0  0       0  0  0       0  0  0      0   

methods                     ...        gcond                     sfgc        \
threshold                   ... 15        5         10        15   5  10 15   
Dataset  Compression ratio  ...                                               
cora     0.125              ...  0  0.000000  0.000000  0.000000    0  0  0   
         0.250              ...  0  0.000000  0.000000  0.000000    0  0  0   
         0.500              ...  0  0.346032  0.535878  0.636795    0  0  0   
citeseer 0.125              ...  0  0.000000  0.000000  0.000000    0  0  0   
         0.250              ...  0  0.542857  0.673985  0.776188    0  0  0   
         0.500              ...  0  0.000000  0.000000  0.000000    0  0  0   

methods                    one-step        
threshold                        5  10 15  
Dataset  Compression ratio                 
cora     0.125                    0  0  0  
         0.250                    0  0  0  
         0.500                    0  0  0  
citeseer 0.125                    0  0  0  
         0.250                    0  0  0  
         0.500                    0  0  0  

[6 rows x 21 columns]

In [109]:
exp.results_store.cosine_1

methods                    random       herding       kcenter       kmeans  \
threshold                      5  10 15      5  10 15      5  10 15     5    
Dataset  Compression ratio                                                   
cora     0.125                  0  0  0       0  0  0       0  0  0      0   
         0.250                  0  0  0       0  0  0       0  0  0      0   
         0.500                  0  0  0       0  0  0       0  0  0      0   
citeseer 0.125                  0  0  0       0  0  0       0  0  0      0   
         0.250                  0  0  0       0  0  0       0  0  0      0   
         0.500                  0  0  0       0  0  0       0  0  0      0   

methods                     ...        gcond                     sfgc        \
threshold                   ... 15        5         10        15   5  10 15   
Dataset  Compression ratio  ...                                               
cora     0.125              ...  0  0.000000  0.000000  0.000000    0  0  0   
         0.250              ...  0  0.000000  0.000000  0.000000    0  0  0   
         0.500              ...  0  0.431086  0.587106  0.644101    0  0  0   
citeseer 0.125              ...  0  0.000000  0.000000  0.000000    0  0  0   
         0.250              ...  0  0.495033  0.542787  0.600435    0  0  0   
         0.500              ...  0  0.000000  0.000000  0.000000    0  0  0   

methods                    one-step        
threshold                        5  10 15  
Dataset  Compression ratio                 
cora     0.125                    0  0  0  
         0.250                    0  0  0  
         0.500                    0  0  0  
citeseer 0.125                    0  0  0  
         0.250                    0  0  0  
         0.500                    0  0  0  

[6 rows x 21 columns]

### Code to generate Benchmark model - COra and Citeseer

In [ ]:
# import matplotlib.pyplot as plt

# class training():
#   def __init__(self, model, data, mask = None):

#     self.model = model
#     self.dataset = data
#     self.criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
#     self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.0001)  # Define optimizer.
#     if mask == None:
#       self.train_old_mask = self.dataset.data.train_mask
#     else:
#       self.train_old_mask = mask
#       self.dataset.data.train_mask = mask
#     self.embed = 0


#   def train(self, data):
#       self.optimizer.zero_grad()  # Clear gradients.
#       out = self.model.forward(data.x, data.edge_index)  # Perform a single forward pass.

#       loss = self.criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.

#       loss.backward()  # Derive gradients.
#       self.optimizer.step()  # Update parameters based on gradients.
#       return loss, out

#   def train_model(self, epo=100):
#     data = self.dataset[0]
#     #print("Shape is', data.x.shape)
#     loss_t = []

#     train_loss = []
#     val_loss = []
#     test_loss = []

#     test_accuracy = []
#     train_accuracy = []
#     val_accuracy = []

#     ep=[]

#     macro_f1_train = []
#     macro_f1_val =[]
#     macro_f1_test=[]

#     print('Training Started...')
#     for epoch in range(1,epo):
#         loss, out = self.train(data)
#         loss_t+=[loss]

#         if epoch % 5 == 0:
#             #visualize_embedding(h, color=data.y, epoch=epoch, loss=loss)
#             ep.append(epoch)

#             with torch.no_grad():

#               train_loss.append(self.criterion(out[data.train_mask], data.y[data.train_mask]).item())
#               val_loss.append(self.criterion(out[data.val_mask], data.y[data.val_mask]).item())
#               test_loss.append(self.criterion(out[data.test_mask], data.y[data.test_mask]).item())

#               train_accuracy.append(((torch.argmax(out[data.train_mask], axis=1) == data.y[data.train_mask]).sum()/len(data.y[data.train_mask])).item()*100)
#               val_accuracy.append(((torch.argmax(out[data.val_mask], axis=1) == data.y[data.val_mask]).sum()/len(data.y[data.val_mask])).item()*100)
#               test_accuracy.append(((torch.argmax(out[data.test_mask], axis=1) == data.y[data.test_mask]).sum()/len(data.y[data.test_mask])).item()*100)

#               # macro_f1_train.append(f1_score(data.y[data.train_mask].cpu(), torch.argmax(out[data.train_mask].cpu(), axis=1), average='macro'))
#               # macro_f1_val.append(f1_score(data.y[data.val_mask].cpu(), torch.argmax(out[data.val_mask].cpu(), axis=1), average='macro'))
#               # macro_f1_test.append(f1_score(data.y[data.test_mask].cpu(), torch.argmax(out[data.test_mask].cpu(), axis=1), average='macro'))
#               #print('Epoch : ', epoch, " Test Accuracy: ", test_accuracy[-1], "MacroF1 train: ",macro_f1_train[-1], "MacroF1 Val: ",macro_f1_val[-1])
#               print(f'Epoch : {epoch:.2f}, Training Accuracy: {train_accuracy[-1]:.2f}, Testing Accuracy: {test_accuracy[-1]:.2f}')
#               #print("train loss", loss.item(), "Val loss: ", val_loss[-1])

#             #time.sleep(0.3)
#     print('Training Finished!')
#     with torch.no_grad():
#       #self.embed = h
#       plt.figure(figsize=(9,6))
#       plt.plot(ep, train_loss)
#       plt.plot(ep, val_loss)
#       plt.plot(ep, test_loss)
#       plt.xlabel("Epoch")
#       plt.ylabel("Loss")
#       plt.grid(True, linestyle='--')
#       plt.legend(['Training Loss','Validation Loss', 'Testing Loss'])
#       plt.title('Loss Evolution')
#       plt.show()
#       #visualize_embedding(self.embed, color=data.y, epoch=epoch, loss=loss)

In [ ]:
# data = Planetoid(root='data', name= "citeseer")
# model_cora = GCNModel(num_features = 3703, hidden_channels = 256, num_classes = 6, num_layers = 2, dropout=0.5, with_bn = False)
# exp = training(model_cora, data)
# exp.train_model(150)


In [ ]:
# PATH = "/content/sample_data/model_citeseer.pt"
# torch.save(exp.model.state_dict(), PATH)

# Fidelity - Since we have a baseline

### TO do:
##### code storage method - Tables for each model seed -5 , Table for each kind of similarity -3  , Tables for average of 5 seeds, What to store: For each threshold, store average similarity across various node samples

##### code display method: calculates average across model seeds for all three similarity and then prints the three table also outputs the csv file.
##### store all the links of files in a datastructure - Key as tuple of dataset/method/compression/seed.

##### Visualization: Explaination of Benchmark Model, compared to model of each methods for two different nodes - High entropy and low entropy. Does visualization and similarity calculations match?


In [ ]:
# import pandas as pd
# methods = ["kmeans", "gcond","herding"]
# threshold = [5,10,15]
# col_idx = pd.MultiIndex.from_product([methods,threshold],names = ["methods","threshold"])

# data = ["cora", "citeseer"]
# compression = [0.125,0.25,0.5]
# row_idx = pd.MultiIndex.from_product([data,compression],names = ["Dataset","Compression ratio"])
# df = pd.DataFrame(columns=col_idx, index=row_idx)
# df

In [ ]:
# df.loc[("cora",0.125),("kmeans")] = [5,15,20]

In [ ]:
# df

In [3]:
# import pandas as pd
# df = pd.read_csv("Avg_Jacard_edges_similarity")
# df.to_csv("avg_jacard_edge.csv")